In [1]:
import tabula
import pandas as pd

In [2]:
# Função para padronizar os valores do IPI
def converter_ipi(x):
    if x == "NT":
        x = "NT"
    elif x is None:
        x = np.nan
    else:
        x = float(x)
    return x

In [3]:
file = 'TIPI.pdf' 
# fonte: http://receita.economia.gov.br/acesso-rapido/legislacao/documentos-e-arquivos/tipi-1.pdf/view
# acessado em 14/07/2020

dfs = tabula.read_pdf(file, pages = 'all', lattice= True)

Got stderr: jul 15, 2020 8:14:13 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMAÇÕES: OpenType Layout tables used in font ArialMT are not implemented in PDFBox and will be ignored
jul 15, 2020 8:14:17 PM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFORMAÇÕES: OpenType Layout tables used in font Arial-BoldMT are not implemented in PDFBox and will be ignored



In [4]:
len(dfs)

408

In [5]:
# Lista o número de colunas de cada DataFrame que o Tabula encontrou
shape_list = []
for df in range(len(dfs)):
    shape_list.append(dfs[df].shape[1])
    
# Altera o nome das colunas de cada DataFrame para uma numeração gernérica (necessário para o append)
for df in range(len(dfs)):
    dfs[df].columns = list(range(shape_list[df]))

In [6]:
# Declara o DataFrame vazio 'tipi' e faz o append de cada DataFrame contido em Dfs
tipi = pd.DataFrame()
for i in range(len(dfs)):
    tipi = tipi.append(dfs[i])

# arruma o indice
tipi = tipi.reset_index(drop=True)

# Deixa somente as primeiras colunas e altera os títulos
tipi = tipi[[0,1,2]]
tipi.columns = ['ncm', 'descricao', 'alq_ipi']

# Temos um único DataFrame com todas as tabelas de dfs
tipi

,ncm,descricao,alq_ipi
0,NaN,TABELA DE INCIDÊNCIA DO IMPOSTO SOBRE PRODUTOS...,NaN
1,NaN,NaN,NaN
2,01.01,"Cavalos, asininos e muares, vivos.",NaN
3,0101.2,- Cavalos:,NaN
4,0101.21.00,--Reprodutores de raça pura,NT
...,...,...,...
16519,9704.00.00,"Selos postais, selos fiscais, marcas postais, ...",NT
16520,NaN,NaN,NaN
16521,9705.00.00,"Coleções e espécimes para coleções, de zoologi...",NT
16522,NaN,NaN,NaN


In [7]:
# Separando os Ex-Tarifários
tipi_ex = tipi.fillna(method = 'ffill')
tipi_ex = tipi_ex[tipi['descricao'].str.startswith('Ex ', na=False)]

# Padronizando o valor das alóquotas de ipi em tipi_ex
ipi_ex = tipi_ex['alq_ipi'].apply(converter_ipi)
tipi_ex['alq_ipi'] = ipi_ex

# Tabela dos Ex-Tarifários
tipi_ex

,ncm,descricao,alq_ipi
185,0210.91.00,Ex 01 - Miudezas; farinhas e pós dessas miudezas,NT
187,0210.92.00,Ex 01 - Miudezas; farinhas e pós dessas miudezas,NT
189,0210.93.00,Ex 01 - Miudezas; farinhas e pós dessas miudezas,NT
198,0210.99.90,Ex 01 – Farinhas e pós das miudezas do código ...,NT
537,0305.71.00,"Ex 01 - De tubarão seco, mesmo salgado mas não...",5
...,...,...,...
16352,9504.90.90,"Ex 02 - Ficha, marca (escore) ou tento",40
16391,9508.10.00,"Ex 01 - Coleções de animais de zoológicos, de ...",0
16506,9619.00.00,"Ex 01 - Artigos de vestuário, de plástico",5
16507,9619.00.00,Ex 02 - Outros artigos de plástico,15


In [8]:
# Limpeza de valores vazios na coluna de NCM
notna = tipi[['ncm']].notna()
tipi = tipi[notna['ncm']]
tipi

,ncm,descricao,alq_ipi
2,01.01,"Cavalos, asininos e muares, vivos.",NaN
3,0101.2,- Cavalos:,NaN
4,0101.21.00,--Reprodutores de raça pura,NT
5,0101.29.00,--Outros,NT
6,0101.30.00,- Asininos,NT
...,...,...,...
16515,9702.00.00,"Gravuras, estampas e litografias, originais.",NT
16517,9703.00.00,Produções originais de arte estatuária ou de e...,NT
16519,9704.00.00,"Selos postais, selos fiscais, marcas postais, ...",NT
16521,9705.00.00,"Coleções e espécimes para coleções, de zoologi...",NT


In [9]:
# Limpeza de textos mais compridos que 10 caracteres (comprimento mínimo do nível mais analítico da NCM)
# e com alíquota vazia
not_ncm = tipi[(tipi['ncm'].str.len() >= 10) & (tipi['alq_ipi'].isna())].index
tipi = tipi.drop(not_ncm)
tipi

,ncm,descricao,alq_ipi
2,01.01,"Cavalos, asininos e muares, vivos.",NaN
3,0101.2,- Cavalos:,NaN
4,0101.21.00,--Reprodutores de raça pura,NT
5,0101.29.00,--Outros,NT
6,0101.30.00,- Asininos,NT
...,...,...,...
16515,9702.00.00,"Gravuras, estampas e litografias, originais.",NT
16517,9703.00.00,Produções originais de arte estatuária ou de e...,NT
16519,9704.00.00,"Selos postais, selos fiscais, marcas postais, ...",NT
16521,9705.00.00,"Coleções e espécimes para coleções, de zoologi...",NT


In [10]:
# Limpeza de textos mais compridos que 3 caracteres 
# (menos as últimas três ocorrências, que constatamos ser um problema na leitura das tabelas)
alqt_clean = tipi[tipi['alq_ipi'].str.len() >3][:-3].index
tipi = tipi.drop(alqt_clean)

In [11]:
# Checagem das descrições vazias
tipi[tipi['descricao'].isna()]

,ncm,descricao,alq_ipi
1954,8,NaN,NaN
8082,prunóidea,NaN,NaN
16218,9306.21.10,NaN,Que contenham produtos químicos ou oleorresina...
16220,9306.21.30,NaN,"Outros, com um ou mais projéteis de elastômeros"
16221,9306.21.90,NaN,Outro


In [12]:
# Checagem das NCM duplicadas
tipi[tipi['ncm'].duplicated(keep=False)]

,ncm,descricao,alq_ipi
12100,8418.2,A,10
12416,8418.2,- Refrigeradores do tipo doméstico:,NaN
15050,8703.22,11,NaN
15055,8703.24,18,NaN
15145,8703.22,"--De cilindrada superior a 1.000 cm3, mas não ...",NaN
15153,8703.24,--De cilindrada superior a 3.000 cm3,NaN


In [13]:
# Ajuste fino dos DataFrames
tipi = tipi.drop([8082,12100,15050,15055])
tipi['descricao'][16218] = tipi['alq_ipi'][16218]
tipi['descricao'][16220] = tipi['alq_ipi'][16220]
tipi['descricao'][16221] = tipi['alq_ipi'][16221]
tipi['alq_ipi'][16218] = 20
tipi['alq_ipi'][16220] = 20
tipi['alq_ipi'][16221] = 20

In [14]:
ipi = tipi['alq_ipi'].apply(converter_ipi)
tipi['alq_ipi'] = ipi

In [15]:
# Verificando se as NCMs são valores únicos
tipi['ncm'].describe()

count       14765
unique      14765
top       7304.41
freq            1
Name: ncm, dtype: object

In [16]:
group_ipi = tipi.groupby('alq_ipi').count().sort_values('ncm',ascending=False)['ncm']
total = group_ipi.sum()
freq_ipi = group_ipi.apply(lambda x : 100* x/total).round(2)
freq_acum = freq_ipi.cumsum().round(1)
group_ipi = pd.DataFrame({'group_ipi' : group_ipi ,'freq_ipi':freq_ipi, 'freq_acum': freq_acum})
print(group_ipi)
print('................................')
print('Total de NCMs Analíticas:', total)
print('................................')

         group_ipi  freq_ipi  freq_acum
alq_ipi                                
0.0           6037     58.58       58.6
5.0           1394     13.53       72.1
NT             736      7.14       79.2
10.0           720      6.99       86.2
15.0           712      6.91       93.1
20.0           207      2.01       95.2
8.0            176      1.71       96.9
2.0             72      0.70       97.6
12.0            68      0.66       98.2
25.0            51      0.49       98.7
30.0            31      0.30       99.0
4.0             22      0.21       99.2
45.0            18      0.17       99.4
22.0            15      0.15       99.5
18.0            13      0.13       99.7
35.0            10      0.10       99.8
40.0             7      0.07       99.8
7.0              4      0.04       99.9
60.0             3      0.03       99.9
13.0             2      0.02       99.9
6.0              2      0.02      100.0
16.0             2      0.02      100.0
3.0              1      0.01      100.0


In [17]:
# Salvando os DataFrames em arquivos csv
tipi.to_csv('tipi.csv', sep=';')
tipi_ex.to_csv('tipi_ex.csv', sep=';')